In [37]:
import pandas as pd
import pandas_profiling
import seaborn as sns
import os
import numpy as np
from IPython.display import display

# pd.set_option("display.max_rows", 999)
# pd.set_option('max_colwidth',100)


In [38]:
filepath = os.path.abspath('./Enron_Kaggle_dataset/email_features.csv')

emails = pd.read_csv(filepath)

headers = [header for header in emails.columns]

print("Successfully loaded {} rows and {} columns!".format(emails.shape[0], emails.shape[1]))

print(display(emails.head()))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd1 in position 2: invalid continuation byte

In [21]:
# checking message of Third file

print(emails.loc[25]["message"])

Message-ID: <19034252.1075855687825.JavaMail.evans@thyme>
Date: Tue, 26 Sep 2000 09:28:00 -0700 (PDT)
From: phillip.allen@enron.com
To: kholst@enron.com
Subject: Investment Structure
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: kholst@enron.com
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 
04:28 PM ---------------------------


"George Richards" <cbpres@austin.rr.com> on 09/26/2000 01:18:45 PM
Please respond to <cbpres@austin.rr.com>
To: "Phillip Allen" <pallen@enron.com>
cc: "Larry Lewter" <retwell@mail.sanmarcos.net>, "Claudia L. Crocker" 
<clclegal2@aol.com> 
Subject: Investment Structure


STRUCTURE:
Typically the structure is a limited partnership with a corporate (or LLC)
general partner.  The General Partner owns 1% of the project and carries the
liability of c

 ### Features and Label
 
Headers such as To, Subject, From and the message body are used as features. The X-Folder contains the label. All the features are in strings will need to be converted into numerical format for the computer to interpret them.

In [22]:
import email

def insert_value(dictionary, key, value):
    if key in dictionary:
        values = dictionary.get(key)
        values.append(value)
        dictionary[key] = values
    else:
        dictionary[key] = [value]
    return dictionary

def get_headers(df, header_names):
    headers = {}
    messages = df["message"]
    for message in messages:
        e = email.message_from_string(message)
        for item in header_names:
            header = e.get(item)
            insert_value(dictionary = headers, key = item, value = header) 
    print("Successfully retrieved header information!")
    return headers
header_names = ["Date", "Subject", "X-Folder", "X-From", "X-To"]    
headers = get_headers(emails, header_names)

Successfully retrieved header information!


In [23]:
# extracting message from data and adding new feature of message_body to data

def get_messages(df):
    messages = []
    for item in df["message"]:
        # Return a message object structure from a string
        e = email.message_from_string(item)    
        # get message body  
        message_body = e.get_payload()
        message_body = message_body.lower()
        messages.append(message_body)
    print("Successfully retrieved message body from e-mails!")
    return messages
msg_body = get_messages(emails)
emails["Message-Body"] = msg_body

Successfully retrieved message body from e-mails!


In [24]:
emails.head(3)

,file,message,Message-Body
0,allen-p/_sent_mail/1.,"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700...",here is our forecast\n\n
1,allen-p/_sent_mail/10.,"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 ...",traveling to have a business meeting takes the fun out of the trip. especially if you have to p...
2,allen-p/_sent_mail/100.,"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700...",test successful. way to go!!!


In [25]:
# adding employee name to data

x_from = pd.DataFrame(headers["X-From"], columns = ["X-From"])
print(x_from.iloc[:1000]["X-From"].unique()[:10])
if "X-From" not in emails.columns:
    emails = pd.concat([emails, x_from], axis = 1, join = 'inner')


['Phillip K Allen' 'Ina Rangel'
 'Multex Investor <1.11913372.-2@multexinvestornetwork.com>'
 '"Bloomberg.com" <messenger@ecm.bloomberg.com>'
 '"Arthur O\'Donnell" <aod@newsdata.com>' 'critical.notice@Enron.com'
 'market-reply@LISTSERV.DOWJONES.COM' 'Rebecca W Cantrell'
 '"Earnings.com" <webmaster@earnings.com>' 'Paul Kaufman']


In [26]:
def add_name(df, column, labels):
    new_data = {}
    for item in df[column]: 
        tokens = item.split('/')
        for i in range(0, len(labels)):
            value = tokens[i]
            key = labels[i]
            new_data = insert_value(new_data, key, value)        
    for key, value in new_data.items():
        df[key] = pd.Series(value)
    print("Successfully added new column!")
    return df
emails = add_name(df = emails, column = "file", labels = ["employee"])

Successfully added new column!


In [27]:
emails.head(3)

,file,message,Message-Body,X-From,employee
0,allen-p/_sent_mail/1.,"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700...",here is our forecast\n\n,Phillip K Allen,allen-p
1,allen-p/_sent_mail/10.,"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 ...",traveling to have a business meeting takes the fun out of the trip. especially if you have to p...,Phillip K Allen,allen-p
2,allen-p/_sent_mail/100.,"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700...",test successful. way to go!!!,Phillip K Allen,allen-p


In [28]:
def add_headers(df, header_list):
    for label in header_list:
        df_new = pd.DataFrame(headers[label], columns = [label])
        if label not in df.columns:
            df = pd.concat([df, df_new], axis = 1)
    return df

remaining_headers = ["Date", "Subject", "X-To", "X-From", "X-Folder"]

emails = add_headers(df = emails, header_list = remaining_headers)

In [29]:
emails.head(3)

,file,message,Message-Body,X-From,employee,Date,Subject,X-To,X-Folder
0,allen-p/_sent_mail/1.,"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700...",here is our forecast\n\n,Phillip K Allen,allen-p,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",,Tim Belden <Tim Belden/Enron@EnronXGate>,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail"
1,allen-p/_sent_mail/10.,"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 ...",traveling to have a business meeting takes the fun out of the trip. especially if you have to p...,Phillip K Allen,allen-p,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail"
2,allen-p/_sent_mail/100.,"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700...",test successful. way to go!!!,Phillip K Allen,allen-p,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,Leah Van Arsdall,\Phillip_Allen_Dec2000\Notes Folders\'sent mail


In [30]:
emails["Date"] = pd.to_datetime(emails["Date"])

In [33]:
subset = emails.loc[np.random.choice(emails.index, 5000, replace=False)]

### Saving subset as csv file:

# subset = emails.head(100000).to_csv('subset_to_test_mongo.csv', index = False)

In [34]:
### Pandas Profiling: 

# subset.profile_report()